# Multi Agent 2


### Patterns 

   - Reflection : 자기 비판
   - Reflexion : 현재 답변의 약점 탐색

##### Pattern 1 : Reflection

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from typing import List, Sequence
from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.graph import END, MessageGraph


In [ ]:
reflection_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 블로그를 평가하는 인플루언서입니다. 사용자의 글에 대한 비평과 추천을 생성하세요."
            "길이, 바이럴율, 스타일 등에 대한 요청을 포함하여 자세한 추천을 제공하세요.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

generation_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 게시물을 작성하는 업무를 맡은 기술 인플루언서의 보조자입니다."
            " 사용자의 요청에 따라 최상의 게시물을 작성하세요."
            " 사용자가 비평을 제공하면, 이전 시도를 수정하여 250개 단어로 답변하세요.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)


llm = ChatOpenAI(model="gpt-4o-mini")
generate_chain = generation_prompt | llm
reflect_chain = reflection_prompt | llm

In [ ]:
REFLECT = "reflect"
GENERATE = "generate"


def generation_node(state: Sequence[BaseMessage]):
    return generate_chain.invoke({"messages": state})


def reflection_node(messages: Sequence[BaseMessage]) -> List[BaseMessage]:
    res = reflect_chain.invoke({"messages": messages})
    return [HumanMessage(content=res.content)]


builder = MessageGraph()
builder.add_node(GENERATE, generation_node)
builder.add_node(REFLECT, reflection_node)
builder.set_entry_point(GENERATE)


def should_continue(state: List[BaseMessage]):
    if len(state) > 6:
        return END
    return REFLECT


builder.add_conditional_edges(GENERATE, should_continue)
builder.add_edge(REFLECT, GENERATE)

graph = builder.compile()


In [ ]:
inputs = HumanMessage(content="""다음의 블로그 내용을 더 멋지게 만들어 주세요:"

            — Agentic AI는 최근 LLM 기반 시스템의 진화된 형태로, **"자율적이고 목표 지향적인 행동을 할 수 있는 AI"**를 말합니다. 

                                  """)
response = graph.invoke(inputs)

print(response)

##### Pattern 2 : Reflexion

In [ ]:
from langchain_core.messages import BaseMessage, ToolMessage
from langgraph.graph import END, MessageGraph
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.utilities.tavily_search import TavilySearchAPIWrapper
from langchain_core.tools import StructuredTool
from langgraph.prebuilt import ToolNode
from langchain_core.output_parsers import JsonOutputToolsParser, PydanticToolsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from typing import List
from pydantic import BaseModel, Field
import datetime


In [ ]:
search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=search, max_results=5)

class Reflection(BaseModel):
    missing: str = Field(description="누락된 부분에 대한 비판")
    superfluous: str = Field(description="불필요한 것에 대한 비판")


class AnswerQuestion(BaseModel):
    """Answer the question."""

    answer: str = Field(description="질문에 대한 답변은 250자 내로 작성")
    reflection: Reflection = Field(description="첫 번째 답변에 대한 평가")
    search_queries: List[str] = Field(
        description="현재 답변에 대한 비판을 해결하기 위한 개선 사항을 조사하기 위한 1~3개의 검색어"
    )


class ReviseAnswer(AnswerQuestion):
    """Revise your original answer to your question."""

    references: List[str] = Field(
        description="업데이트된 답변에 대한 동기를 인용"
    )

def run_queries(search_queries: list[str], **kwargs):
    """Run the generated queries."""
    return tavily_tool.batch([{"query": query} for query in search_queries])


tool_node = ToolNode(
    [
        StructuredTool.from_function(run_queries, name=AnswerQuestion.__name__),
        StructuredTool.from_function(run_queries, name=ReviseAnswer.__name__),
    ]
)

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini")
parser = JsonOutputToolsParser(return_id=True)

actor_prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """당신은 전문적인 연구자입니다.
                Current time: {time}

                1. {first_instruction}
                2. 답변을 되돌아보고 비판하세요. 개선을 극대화하기 위해 진지하게 비판하세요.
                3. 정보를 확인하고 답변을 개선하기 위해 검색어를 추천하세요""",
        ),
        MessagesPlaceholder(variable_name="messages"),
        ("system", "사용자의 질문에 필요한 형식을 사용하여 답변하세요."),
    ]
).partial(
    time=lambda: datetime.datetime.now().isoformat(),
)


first_responder = actor_prompt_template.partial(
    first_instruction="250자 내로 자세한 답변을 하세요."
) | llm.bind_tools(tools=[AnswerQuestion], tool_choice="AnswerQuestion")
validator = PydanticToolsParser(tools=[AnswerQuestion])


revise_instructions = """새로운 정보를 사용하여 이전 답변을 수정하세요.
                        - 이전 비평을 활용하여 답변에 중요한 정보를 추가하세요.
                            - 수정된 답변의 검증을 위해 반드시 인용 번호를 포함하세요.
                            - 답변 하단에 "참고문헌" 섹션을 추가하세요(참고문헌은 단어 제한에 포함되지 않습니다). 다음 형식으로 작성하세요:
                                - [1] https://example.com
                                - [2] https://example.com
                        - 이전 비평을 활용하여 답변에서 불필요한 정보를 제거하고 250 단어를 넘지 않도록 작성하세요.
"""


revisor = actor_prompt_template.partial(
    first_instruction=revise_instructions
) | llm.bind_tools(tools=[ReviseAnswer], tool_choice="ReviseAnswer")

In [ ]:
MAX_ITERATIONS = 2

builder = MessageGraph()
builder.add_node("draft", first_responder)
builder.add_node("execute_tools", tool_node)
builder.add_node("revise", revisor)
builder.add_edge("draft", "execute_tools")
builder.add_edge("execute_tools", "revise")


def event_loop(state: List[BaseMessage]) -> str:
    count_tool_visits = sum(isinstance(item, ToolMessage) for item in state)
    num_iterations = count_tool_visits
    if num_iterations > MAX_ITERATIONS:
        return END
    return "execute_tools"


builder.add_conditional_edges("revise", event_loop)
builder.set_entry_point("draft")
graph = builder.compile()

# print(graph.get_graph().draw_mermaid())
# print(graph.get_graph().draw_ascii())

graph.get_graph().draw_mermaid_png(output_file_path="graph.png")

res = graph.invoke(
    "NPU에 대한 고충 영역을 식별하고 이를 해결하기 위해 자본 투자를 유치한 스타트업 목록을 나열하세요."
)
print(res[-1].tool_calls[0]["args"]["answer"])
print(res)